In [1]:
import pyspark
from delta import configure_spark_with_delta_pip

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("Delta Lake Querying")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

24/12/12 19:37:28 WARN Utils: Your hostname, emif-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.136 instead (on interface en0)
24/12/12 19:37:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/emif/Documents/mutt/test-dbt-spark/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/emif/.ivy2/cache
The jars for the packages stored in: /Users/emif/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e06ffd0c-da25-4d8f-bfe6-855dcf376b4d;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.1 in central
	found io.delta#delta-storage;3.2.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 277ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.1 from central in [default]
	io.delta#delta-storage;3.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [3]:
BASE_PATH = "../dbt_pipeline/spark-warehouse"

In [4]:
customers_path = f"{BASE_PATH}/customers/"
stg_customers = f"{BASE_PATH}/stg_customers/"
dim_customers = f"{BASE_PATH}/dim_customers/"


# Read the Delta tables
df_customers = spark.read.format("delta").load(customers_path)
df_stg_customers = spark.read.format("delta").load(stg_customers)
df_dim_customers = spark.read.format("delta").load(dim_customers)

In [5]:
df_customers.printSchema()
df_stg_customers.printSchema()
df_dim_customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- valid_from: date (nullable = true)
 |-- data: string (nullable = true)
 |-- source_filename: string (nullable = true)
 |-- loaded_at: timestamp (nullable = true)

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- tax_id: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- loyalty_segment: string (nullable = true)

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)



In [6]:
df_customers.show(5, truncate=True)

24/12/12 19:37:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+----------+--------------------+--------------------+--------------------+
|customer_id|valid_from|                data|     source_filename|           loaded_at|
+-----------+----------+--------------------+--------------------+--------------------+
|   29613599|      NULL|{"tax_id":"217417...|/databricks-datas...|2024-03-07 20:42:...|
|   33374418|      NULL|{"tax_id":"292667...|/databricks-datas...|2024-03-07 20:42:...|
|   18744161|      NULL|{"tax_id":"407488...|/databricks-datas...|2024-03-07 20:42:...|
|   27126846|      NULL|{"tax_id":"829357...|/databricks-datas...|2024-03-07 20:42:...|
|    2907473|      NULL|{"tax_id":"240370...|/databricks-datas...|2024-03-07 20:42:...|
+-----------+----------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [7]:
df_stg_customers.show(5, truncate=True)

+-----------+--------------------+-----------+-----+---------+---------------+
|customer_id|       customer_name|     tax_id|state|     city|loyalty_segment|
+-----------+--------------------+-----------+-----+---------+---------------+
|   15923408|        excel sports|814025412.0|   NY| Perinton|              1|
|   17892598| jacklynpresents.com|249757266.0|   NY|    Utica|              1|
|   52481342|search financial ...|607148555.0|   NC|CHARLOTTE|              3|
|   15632773|nbs commercial in...|721659808.0|   FL|     NULL|              0|
|   17472628|custom design con...|196316204.0|   OH|  WILLARD|              0|
+-----------+--------------------+-----------+-----+---------+---------------+
only showing top 5 rows



In [8]:
df_dim_customers.show(5, truncate=True)


+-----------+--------------------+
|customer_id|       customer_name|
+-----------+--------------------+
|    8520504|judith gurley pla...|
|    5580101|            allsides|
|    9296498|executive studies...|
|     635244|renner and compan...|
|   62484883|upward consulting...|
+-----------+--------------------+
only showing top 5 rows

